In [1]:
import re
import jieba
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import joblib


# 分词
def tokenize(text):
    """
    带有语料清洗功能的分词函数
    """
    text = re.sub("\{%.+?%\}", " ", text)  # 去除 {%xxx%} (地理定位, 微博话题等)
    text = re.sub("@.+?( |$)", " ", text)  # 去除 @xxx (用户名)
    text = re.sub("【.+?】", " ", text)  # 去除 【xx】 (里面的内容通常都不是用户自己写的)
    icons = re.findall("\[.+?\]", text)  # 提取出所有表情图标
    text = re.sub("\[.+?\]", "IconMark", text)  # 将文本中的图标替换为`IconMark`

    tokens = []
    for k, w in enumerate(jieba.lcut(text)):
        w = w.strip()
        if "IconMark" in w:  # 将IconMark替换为原图标
            for i in range(w.count("IconMark")):
                tokens.append(icons.pop(0))
        elif w and w != '\u200b' and w.isalpha():  # 只保留有效文本
            tokens.append(w)
    return tokens


pp = './data'
# 加载模型
model_path0 = pp+'/bayes_01.02.pkl'
model0 = joblib.load(model_path0)
# 加载词典
vec_path0 = pp+'/feature_01.02.pkl'
vec0 = CountVectorizer(decode_error="replace", vocabulary=pickle.load(open(vec_path0, "rb")))


# 处理数据
def getType(string_l):
    type_=['ICT', '新能源汽车', '生物医药', '医疗器械', '钢铁', 
       '能源', '工业机器人', '先进轨道交通', '数控机床',  '工业软件', 
       '高端装备', '半导体', '人工智能', '稀土']
    
    if isinstance(string_l,str):
        string_l=[string_l]
    if isinstance(string_l,list):
        X_data = []
        for string in string_l:
            X_data.append(" ".join(tokenize(string)))

        vecc = vec0.transform(X_data)
        result_pre = model0.predict(vecc)
        result_pre_proba = model0.predict_proba(vecc)
        
        res=[]
        for i in range(len(string_l)):
            sin_res=[]
            end_index=result_pre[i]
            end_score=result_pre_proba[i][end_index]
            if end_score<0.75:
                sin_res.append('其他')
                sin_res.append(-1)
            else:
                sin_res.append(type_[end_index])
                sin_res.append(end_score)
            sin_res.append([type_[j]+':'+str(result_pre_proba[i][j]) for j in range(len(type_))])
            res.append(sin_res)
        return res
    else:
        print('error')


if __name__ == '__main__':
    string = ["妈了个逼的","你是不是傻子"]
    result=getType(string)
    print(result)
    all_res=[i[0] for i in result]
    print(all_res)



Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/data/fuwen/anaconda3/envs/sw0/lib/python3.7/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpys4ospoi' -> '/tmp/jieba.cache'
Loading model cost 0.747 seconds.
Prefix dict has been built successfully.


[['其他', -1, ['ICT:0.07311546840958608', '新能源汽车:0.21925925925925946', '生物医药:0.05333333333333336', '医疗器械:0.021699346405228782', '钢铁:0.030588235294117676', '能源:0.09542483660130718', '工业机器人:0.2630065359477121', '先进轨道交通:0.023267973856209163', '数控机床:0.0366884531590414', '工业软件:0.013246187363834431', '高端装备:0.018649237472766887', '半导体:0.008714596949891075', '人工智能:0.012810457516339873', '稀土:0.13019607843137262']], ['其他', -1, ['ICT:0.07311546840958608', '新能源汽车:0.21925925925925946', '生物医药:0.05333333333333336', '医疗器械:0.021699346405228782', '钢铁:0.030588235294117676', '能源:0.09542483660130718', '工业机器人:0.2630065359477121', '先进轨道交通:0.023267973856209163', '数控机床:0.0366884531590414', '工业软件:0.013246187363834431', '高端装备:0.018649237472766887', '半导体:0.008714596949891075', '人工智能:0.012810457516339873', '稀土:0.13019607843137262']]]
['其他', '其他']
